# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [ ]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [ ]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(mapbox_token)

# Import Data

In [ ]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [ ]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    # Calculate the mean number of housing units per year (hint: use groupby) 
    sfo_unit_ave = sfo_data['housing_units'].groupby(['year']).mean()
    
    fig_housing_units = sfo_unit_ave.hvplot.bar(
        xlabel='Year',
        ylabel='Housing Units',
        rot=45,
        ylim=(370000, 385000),
        yformatter='%0f',
        title='Housing Units in San Francisco from 2010 to 2016')

    return fig_housing_units

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    # Calculate the average gross rent and average sale price per square foot
    sfo_ave_spsf = sfo_data[{'sale_price_sqr_foot', 'gross_rent'}].groupby(['year']).mean()

    # re-organizing the columns to match order in starter code:
    sfo_ave_spsf.reindex(columns=['sale_price_sqr_foot', 'gross_rent'])
    
    # Plot the Average Gross Rent per Year as a Line Chart 
    ave_gross_rent = sfo_ave_spsf['gross_rent'].hvplot.line(
        xlabel='Year',
        ylabel='Gross Rent',
        rot=45,
        ylim=(1000, 4500),
        title='Average Gross Rent in San Francisco')
    
    return ave_gross_rent

def average_sales_price():
    """Average Sales Price Per Year."""
    
    # Calculate the average gross rent and average sale price per square foot
    sfo_ave_spsf = sfo_data[{'sale_price_sqr_foot', 'gross_rent'}].groupby(['year']).mean()
    
    # Plot the Average Sales Price per Year as a line chart
    ave_sales_price = sfo_ave_spsf['sale_price_sqr_foot'].hvplot.line(
        xlabel='Year',
        ylabel='Average Sale Price',
        rot=45,
        ylim=(300, 750),
        title='Average Sales Price Per Square Foot in San Francisco')

    return ave_sales_price
    
def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    # Group by year and neighborhood and then create a new dataframe of the mean values
    sfo_neigh_mean = sfo_data[{'sale_price_sqr_foot', 'neighborhood'}].groupby(['year','neighborhood']).mean()

    #sfo_neigh_mean.reset_index(inplace=True)
    sfo_neigh_mean.reset_index(inplace=True)
    
    # Use hvplot to create an interactive line chart of the average price per sq ft.
    sfo_plot = sfo_neigh_mean.hvplot.line(
        x='year', 
        y='sale_price_sqr_foot', 
        width=600, 
        groupby='neighborhood', 
        widget_location='left_top',
        title='Average Price by Neighborhood')

    return sfo_plot
    
def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    # Getting the data from the top 10 expensive neighborhoods
    # Creating dataframe with sale price per sq foot and neighborhood columns:
    sfo_neigh_ave = sfo_data.groupby(['neighborhood']).mean()

    # Sorting the dataframe in descending order to easily identify the top 10 most expensive:
    sfo_neigh_ave.sort_values(by=['sale_price_sqr_foot'], ascending=False, inplace=True)

    # Assigning the Top 10 most expensive on average to new dataframe/variable:
    top_10 = sfo_neigh_ave.head(10)
    top_10.reset_index()

    # Plotting the data from the top 10 expensive neighborhoods
    top_10_plot = top_10[{'sale_price_sqr_foot'}].hvplot.bar(rot=45, title='Top 10 Most Expensive Neighborhoods on Average')
    
    return top_10_plot

def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    # Getting the data from the top 10 expensive neighborhoods
    # Creating dataframe with sale price per sq foot and neighborhood columns:
    sfo_neigh_ave = sfo_data.groupby(['neighborhood']).mean()

    # Sorting the dataframe in descending order to easily identify the top 10 most expensive:
    sfo_neigh_ave.sort_values(by=['sale_price_sqr_foot'], ascending=False, inplace=True)

    # Assigning the Top 10 most expensive on average to new dataframe/variable:
    top_10 = sfo_neigh_ave.head(10)
    
    # resetting the index to prepare to incorporate into parallel coordinates & categories plots: 
    top_10.reset_index(inplace=True)

    # Parallel Coordinates Plot
    top_10_coor = px.parallel_coordinates(
    top_10,
    color= 'sale_price_sqr_foot')

    return top_10_coor

def parallel_categories():
    """Parallel Categories Plot."""
    
    # Getting the data from the top 10 expensive neighborhoods
    # Creating dataframe with sale price per sq foot and neighborhood columns:
    sfo_neigh_ave = sfo_data.groupby(['neighborhood']).mean()

    # Sorting the dataframe in descending order to easily identify the top 10 most expensive:
    sfo_neigh_ave.sort_values(by=['sale_price_sqr_foot'], ascending=False, inplace=True)

    # Assigning the Top 10 most expensive on average to new dataframe/variable:
    top_10 = sfo_neigh_ave.head(10)
    
    # resetting the index to prepare to incorporate into parallel coordinates & categories plots: 
    top_10.reset_index(inplace=True)
    
    # Parallel Categories Plot:
    top_10_cate = px.parallel_categories(
        round(top_10,2),
        dimensions=['neighborhood','sale_price_sqr_foot', 'housing_units', 'gross_rent'],
        color= 'sale_price_sqr_foot',
        color_continuous_scale=px.colors.sequential.Inferno,
        labels={
            "neighborhood": "Neighborhood",
            "sale_price_sqr_foot": "Sale Price Per Sq Foot",
            "housing_units": "Housing Units",
            "gross_rent": "Gross Rent"        
        },
    )

    return top_10_cate
    
def neighborhood_map():
    """Neighborhood Map"""
    
    # Creating dataframe with sale price per sq foot and neighborhood columns:
    sfo_neigh_ave = sfo_data.groupby(['neighborhood']).mean()
    
    # Calculate the mean values for each neighborhood
    # Already calculated mean in df sfo_neigh_ave so joined with location df on index 'neighborhood':
    join_df = sfo_neigh_ave.join(df_neighborhood_locations.set_index('Neighborhood'))

    # resetting the index so it can feed into the mapbox plot:
    join_df.reset_index(inplace=True)
    join_df.head()

    # Create a scatter mapbox to analyze neighborhood info
    map_plot = px.scatter_mapbox(
        round(join_df,2),
        title='Average Sale Price Per Square Foot and Gross Rent in San Francisco',
        lat="Lat",
        lon="Lon",
        size='sale_price_sqr_foot',
        color="gross_rent",
        hover_name='neighborhood',
        zoom=10
    )

    return map_plot

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# grouping the average plots by column and assigning to variable to be input into panel, tab function:
average_plots_column = pn.Column("## Averages Data Plots", average_gross_rent(), average_sales_price())

# grouping the parallel plots by column and assigning to variable to be input into panel, tab function:
parallel_plots = pn.Column('## Parallel Comparison Plots', parallel_coordinates(), parallel_categories())

tabs = pn.Tabs(
    ("Average Plots", average_plots_column),
    ("Interactive Plot", average_price_by_neighborhood()),
    ("Top Most Expensive Neighborhoods", top_most_expensive_neighborhoods()),
    ("Parallel Plots", parallel_plots),
    ("Neighbhorhood Map", neighborhood_map()))

## Serve the Panel Dashboard

In [ ]:
tabs.servable()